# Implement behavior upsampling

In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import json
import numpy as np
import pandas as pd
import src.paths
from src.data import load_behavior

In [3]:
with open(src.paths.REF / 'date_recoffset.json', 'r') as f:
    offsets = json.load(f)
with open(src.paths.REF / 'date_recid.json', 'r') as f:
    recids = json.load(f)

In [4]:
dates = [
    '2022-07-27',
    '2022-07-28',
    '2022-08-01',
    '2022-08-03',
    '2022-08-04',
    '2022-08-06',
    '2022-08-08',
    '2022-08-11',
    '2022-08-12',
    '2022-08-13',
    '2022-08-15',
    '2022-09-12',
    '2022-09-14',
    '2022-09-16',
]

In [43]:
date = dates[0]

In [ ]:
def upsample_behavior(date):
    recid = recids[date]
    lfpstart, lfpend = offsets[date]

    df = load_behavior(recid)
    lfplen = (lfpend - lfpstart + 1) // 2
    lfp_t = pd.date_range(start=0, periods=lfplen, freq='0.4ms')
    behav_t = pd.date_range(start=0, periods=len(df), freq='10ms')

    df = df.set_index(behav_t)
    df = df.reindex(lfp_t).interpolate(method='linear')
    df.index = pd.Series(np.arange(len(df)) / 2500, name='time')
    return df